In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import _init_paths

import os

import torch
import torch.utils.data
from opts import opts
from models.model import create_model, load_model, save_model
from models.data_parallel import DataParallel
from logger import Logger
from datasets.dataset_factory import get_dataset
from trains.train_factory import train_factory
import cv2
import sys
%matplotlib inline
import matplotlib.pyplot as plt

In [6]:
sys.argv = ['-f', 'ctdet', '--exp_id', 'coco_dlav7','--num_workers', '0','--batch_size','2', '--arch','dla_34'] #cmd = 'ctdet --exp_id coco_dla --batch_size 4 --lr 1.25e-4  --gpus 0 --num_workers 0'
opt = opts().parse()
print(opt.task)
print(opt.num_workers)
print(opt.batch_size)

Fix size testing.
training chunk_sizes: [2]
The output will be saved to  C:\Users\avata\FewShot-CenterNet\src\lib\..\..\exp\ctdet\coco_dlav7
ctdet
0
2


In [8]:
torch.manual_seed(opt.seed)
torch.backends.cudnn.benchmark = not opt.not_cuda_benchmark and not opt.test

In [9]:
Dataset = get_dataset(opt.dataset, opt.task)
opt = opts().update_dataset_info_and_set_heads(opt, Dataset)
logger = Logger(opt)
os.environ['CUDA_VISIBLE_DEVICES'] = opt.gpus_str
opt.device = torch.device('cuda' if opt.gpus[0] >= 0 else 'cpu')

heads {'hm': 80, 'wh': 2, 'reg': 2}


In [10]:
model = create_model(opt.arch, opt.heads, opt.head_conv)
optimizer = torch.optim.Adam(model.parameters(), opt.lr)
start_epoch = 0
if opt.load_model != '':
    model, optimizer, start_epoch = load_model(model, opt.load_model, optimizer, opt.resume, opt.lr, opt.lr_step)

In [11]:
Trainer = train_factory[opt.task]
trainer = Trainer(opt, model, optimizer)
trainer.set_device(opt.gpus, opt.chunk_sizes, opt.device)

In [12]:
train_loader = torch.utils.data.DataLoader(
      Dataset(opt, 'train'), 
      batch_size=opt.batch_size, 
      shuffle=True,
      num_workers=opt.num_workers,
      pin_memory=True,
      drop_last=True
  )

==> initializing coco 2017 train data.
loading annotations into memory...
Done (t=20.58s)
creating index...
index created!
Loaded train 118287 samples


In [13]:
img = next(iter(train_loader))

In [14]:
print(img.keys())
print(img['input'].size())

dict_keys(['input', 'hm', 'reg_mask', 'ind', 'wh', 'reg'])
torch.Size([2, 3, 512, 512])


In [15]:
model.to('cuda')
torch.cuda.empty_cache()
outputs = model(img['input'].to('cuda'))
output = outputs[0]
print(output.keys())

dict_keys(['hm', 'wh', 'reg'])


In [17]:
batch=img
mask = batch['reg_mask']
ind = batch['ind']
target_wh = batch['wh']
target_reg = batch['reg']

In [24]:
pred_hm = output['hm'].sigmoid()
print(pred_hm.size())
gt_hm = batch['hm']
print(gt_hm.size())

torch.Size([2, 80, 128, 128])
torch.Size([2, 80, 128, 128])


In [28]:
pos_inds = gt_hm.eq(1)
neg_inds = gt_hm.lt(1)

In [51]:
neg_weights = torch.pow(1 - gt_hm[neg_inds], 4).to('cuda')
print(neg_weights.size())

loss = 0
pos_pred = pred_hm[pos_inds].to('cuda')
neg_pred = pred_hm[neg_inds].to('cuda')
print(neg_pred.size())

pos_loss = torch.log(pos_pred) * torch.pow(1 - pos_pred, 2)
neg_loss = torch.log(1 - neg_pred) * torch.pow(neg_pred, 2) * neg_weights
print(neg_loss.size())

num_pos  = pos_inds.float().sum()
print(num_pos)
pos_loss = pos_loss.sum()
neg_loss = neg_loss.sum()
print(neg_loss)
if pos_pred.nelement() == 0:
    loss = loss - neg_loss
else:
    loss = loss - (pos_loss + neg_loss) / num_pos
print(loss)

torch.Size([2621430])
torch.Size([2621430])
torch.Size([2621430])
tensor(10.)
tensor(-3037.0044, device='cuda:0', grad_fn=<SumBackward0>)
tensor(305.7274, device='cuda:0', grad_fn=<RsubBackward1>)


In [52]:
neg_weights = torch.pow(1 - gt_hm[neg_inds], 4).to('cuda')
print(neg_weights.size())

loss = 0
pos_pred = gt_hm[pos_inds].to('cuda')
neg_pred = gt_hm[neg_inds].to('cuda')
print(neg_pred.size())

pos_loss = torch.log(pos_pred) * torch.pow(1 - pos_pred, 2)
neg_loss = torch.log(1 - neg_pred) * torch.pow(neg_pred, 2) * neg_weights
print(neg_loss.size())

num_pos  = pos_inds.float().sum()
print(num_pos)
pos_loss = pos_loss.sum()
neg_loss = neg_loss.sum()
print(neg_loss)
if pos_pred.nelement() == 0:
    loss = loss - neg_loss
else:
    loss = loss - (pos_loss + neg_loss) / num_pos
print(loss)

torch.Size([2621430])
torch.Size([2621430])
torch.Size([2621430])
tensor(10.)
tensor(-4.0204, device='cuda:0')
tensor(0.4020, device='cuda:0')


In [70]:
img_pred = pred_hm.detach().cpu().numpy() #2x80x180x180
img_pred = img_pred.sum(1)/80
plt.imshow(img_pred[1], cmap='hot', interpolation='nearest')

In [71]:
img_gt = gt_hm.detach().cpu().numpy() #2x80x180x180
img_gt = img_gt.sum(1)/img_gt.max()
plt.imshow(img_gt[1], cmap='hot', interpolation='nearest')

In [72]:
1 in img_gt[1]

True

In [77]:
pred_hm = output['hm'].sigmoid()
print(pred_hm.size())
gt_hm = batch['hm']
print(gt_hm.size())

pred_hm = pred_hm.sum(1)/pred_hm.max()
print(pred_hm.size())
gt_hm = gt_hm.sum(1)/gt_hm.max()
print(gt_hm.size())


pos_inds = gt_hm.eq(1)
neg_inds = gt_hm.lt(1)

neg_weights = torch.pow(1 - gt_hm[neg_inds], 4).to('cuda')
print(neg_weights.size())

loss = 0
pos_pred = pred_hm[pos_inds].to('cuda')
neg_pred = pred_hm[neg_inds].to('cuda')
print(neg_pred.size())

pos_loss = torch.log(pos_pred) * torch.pow(1 - pos_pred, 2)
neg_loss = torch.log(1 - neg_pred) * torch.pow(neg_pred, 2) * neg_weights
print(neg_loss.size())

num_pos  = pos_inds.float().sum()
print(num_pos)
pos_loss = pos_loss.sum()
neg_loss = neg_loss.sum()
print(neg_loss)
if pos_pred.nelement() == 0:
    loss = loss - neg_loss
else:
    loss = loss - (pos_loss + neg_loss) / num_pos
print(loss)

torch.Size([2, 80, 128, 128])
torch.Size([2, 80, 128, 128])
torch.Size([2, 128, 128])
torch.Size([2, 128, 128])
torch.Size([32758])
torch.Size([32758])
torch.Size([32758])
tensor(10.)
tensor(nan, device='cuda:0', grad_fn=<SumBackward0>)
tensor(nan, device='cuda:0', grad_fn=<RsubBackward1>)


In [78]:
pred_hm = output['hm'].sigmoid()
print(pred_hm.size())
gt_hm = batch['hm']
print(gt_hm.size())
pred_hm = gt_hm

pred_hm = pred_hm.sum(1)/pred_hm.max()
print(pred_hm.size())
gt_hm = gt_hm.sum(1)/gt_hm.max()
print(gt_hm.size())


pos_inds = gt_hm.eq(1)
neg_inds = gt_hm.lt(1)

neg_weights = torch.pow(1 - gt_hm[neg_inds], 4).to('cuda')
print(neg_weights.size())

loss = 0
pos_pred = gt_hm[pos_inds].to('cuda')
neg_pred = gt_hm[neg_inds].to('cuda')
print(neg_pred.size())

pos_loss = torch.log(pos_pred) * torch.pow(1 - pos_pred, 2)
neg_loss = torch.log(1 - neg_pred) * torch.pow(neg_pred, 2) * neg_weights
print(neg_loss.size())

num_pos  = pos_inds.float().sum()
print(num_pos)
pos_loss = pos_loss.sum()
neg_loss = neg_loss.sum()
print(neg_loss)
if pos_pred.nelement() == 0:
    loss = loss - neg_loss
else:
    loss = loss - (pos_loss + neg_loss) / num_pos
print(loss)

torch.Size([2, 80, 128, 128])
torch.Size([2, 80, 128, 128])
torch.Size([2, 128, 128])
torch.Size([2, 128, 128])
torch.Size([32758])
torch.Size([32758])
torch.Size([32758])
tensor(10.)
tensor(-4.0528, device='cuda:0')
tensor(0.4053, device='cuda:0')
